# Домашняя работа 1. Рекомендации на основе содержания

- Использовать dataset MovieLens
- Построить рекомендации (регрессия, предсказываем оценку) на фичах:
 - TF-IDF на тегах и жанрах
 - Средние оценки (+ median, variance, etc.) пользователя и фильма
- Оценить RMSE на тестовой выборке

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

In [2]:
links = pd.read_csv('../lecture-1/ml-latest-small/links.csv')
movies = pd.read_csv('../lecture-1/ml-latest-small/movies.csv')
ratings = pd.read_csv('../lecture-1/ml-latest-small/ratings.csv')
tags = pd.read_csv('../lecture-1/ml-latest-small/tags.csv')

## Формирование обучающей выборки для рекоммендаций

TF (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова $ t_{i} $ в пределах отдельного документа.

$${\displaystyle \mathrm {tf} (t,d)={\frac {n_{t}}{\sum _{k}n_{k}}}},$$ где ${\displaystyle n_{t}}$ есть число вхождений слова ${\displaystyle t}$ в документ, а в знаменателе — общее число слов в данном документе.


IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции. Основоположником данной концепции является Карен Спарк Джонс. Учёт IDF уменьшает вес широкоупотребительных слов. Для каждого уникального слова в пределах конкретной коллекции документов существует только одно значение IDF.

$${\mathrm {idf} (t,D)=\log {\frac {|D|}{|\{\,d_{i}\in D\mid t\in d_{i}\,\}|}}}$$ ,
где

|D| — число документов в коллекции;
${\displaystyle |\{\,d_{i}\in D\mid t\in d_{i}\,\}|}$ — число документов из коллекции  D, в которых встречается t (когда ${\displaystyle n_{t}\neq 0}$).

In [3]:
# Добавляем в датасет теги для обработки информации
movies_with_tags = movies.join(tags.groupby('movieId').apply(lambda x: "|".join(x['tag'])).rename('tags'),
            on='movieId', how='left', sort=False)
movies_with_tags = movies_with_tags.fillna('')

In [4]:
movie_count_raitings = movies[['movieId']].join(ratings.groupby('movieId')[['userId']].count(), on='movieId')

In [5]:
# Средние оценки
df_ratings = ratings.groupby('movieId').agg({'userId': np.count_nonzero, 
                                'rating': [np.median, np.var, np.average]})
df_ratings.columns=['userid_count', 'rating_median', 'rating_var', 'rating_average']

movies_ratings = movies_with_tags.join(df_ratings, on='movieId', how='left', sort=False)
movies_ratings = movies_ratings.fillna(0)

In [6]:
movies_ratings['norm_raiting'] = movies_ratings.apply(lambda row: (row['rating_average'])
                             /(movies_ratings['rating_average'].max() - movies_ratings['rating_average'].min())
                 , axis=1)
movies_ratings['norm_userid'] = movies_ratings.apply(lambda row: (row['userid_count'])
                             /(movies_ratings['userid_count'].max() - movies_ratings['userid_count'].min())
                 , axis=1)
movies_ratings['TARGET'] = movies_ratings['norm_userid'] * movies_ratings['norm_raiting']

In [7]:
movies_ratings_ml = movies_ratings[['movieId', 'genres', 'tags', 'userid_count', 'TARGET']]

In [8]:
genres = []
for i in movies_ratings_ml.genres.str.split('|'):
    for j in i:
        genres.append(j)

dict_genres_idf = {i:np.log(len(movies_ratings_ml)/genres.count(i)) for i in genres}

In [9]:
# Выберем только те тэги, которые встречаются больше 5 раз
tags = []
for i in movies_ratings_ml[(movies_ratings_ml.tags.isna()==False)].tags.str.split('|'):
    for j in i :
        tags.append(j)

dict_tags_idf = {i:tags.count(i) for i in tags if tags.count(i)>5 and i!=''}

In [10]:
len(sorted(dict_tags_idf.items(), key=lambda kv: kv[1]) )

125

In [11]:
for i in dict_genres_idf:
    movies_ratings_ml['tf_idf_'+i] = movies_ratings_ml.apply(lambda row: 
                                   (1/len(row['genres'].split('|')))*dict_genres_idf[i]
                                   if i in row['genres'] else 0, axis=1)

/Users/akirichenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
for i in dict_tags_idf:
    movies_ratings_ml['tf_idf_'+i] = movies_ratings_ml.apply(lambda row: 
                                   (1/len(row['tags'].split('|')))*dict_tags_idf[i]
                                   if i in row['tags'] else 0, axis=1)

## Формирование обучающей и тестовой выборки

In [13]:
from sklearn.model_selection import train_test_split

In [30]:
X = movies_ratings_ml.drop(columns=['genres', 'tags', 'TARGET'])
y = movies_ratings_ml['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=4)

## Линейная регрессия для предсказания оценки пользователя

Используем линейную регрессию, чтобы предсказать переменную - среднюю оценку пользователей (rating_average)

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error, mean_squared_error

In [32]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [33]:
predictions = model.predict(X_test)

In [34]:
mean_squared_error(predictions, y_test)

4.0691487649457206e-05

In [35]:
X_test['predictions'] = predictions

/Users/akirichenko/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
results = X_test.merge(movies, how='left', on='movieId')[['movieId', 'predictions', 'title']]
results.sort_values('predictions', ascending=False).head(20)

,movieId,predictions,title
2344,356,0.760598,Forrest Gump (1994)
3210,2571,0.747612,"Matrix, The (1999)"
2038,318,0.725079,"Shawshank Redemption, The (1994)"
1201,593,0.642262,"Silence of the Lambs, The (1991)"
1748,110,0.545664,Braveheart (1995)
2517,480,0.539450,Jurassic Park (1993)
863,589,0.507731,Terminator 2: Judgment Day (1991)
622,47,0.500407,Seven (a.k.a. Se7en) (1995)
2494,2858,0.464661,American Beauty (1999)
781,858,0.457741,"Godfather, The (1972)"
